    - Onderstaande query af en toe draaien om te checken op nieuwe teams vanuit bron FD_STATS

In [5]:
import requests
import pandas as pd
import sqlalchemy
import psycopg2
import datetime

# connection variables
URL = 'football-fortunes.cj5o1qrvsyia.eu-west-1.rds.amazonaws.com'
database_name = 'postgres'
port = 5432
username = 'master'
ww = 'wachtwoord'

# Create SQL Alchemy database connection
db_url = f"postgresql://{username}:{ww}@{URL}:{port}/{database_name}"
engine = sqlalchemy.create_engine(db_url)
sqal_connection = engine.connect()

# Create Psychop connection
psy_connection = psycopg2.connect(
    host = URL,
    port = port,
    database = database_name,
    user= username,
    password = ww
)
psy_cursor = psy_connection.cursor()

# Haal alle teams op in fd_stats en in oddapi, alsmede de teams die al bestaan in tabel teams
teams_fd_stats = pd.read_sql("""SELECT distinct(hometeam) 
                                FROM dsa.fd_stats
                                UNION
                                SELECT distinct(awayteam)
                                FROM dsa.fd_stats
                             ;""", sqal_connection)
bestaande_teams = pd.read_sql('SELECT * FROM dwa.d_teams;', sqal_connection)

# voeg alle teams in obv fd_stats als ze nog niet voorkomen
# in de loop team_fd_stats ophalen, want er staan dubbele records in fd_stats
print(teams_fd_stats)
print(bestaande_teams['team_fd'].tolist())
aantal_nieuw = 0
for i, team in teams_fd_stats.iterrows():
    if team['hometeam'] not in bestaande_teams['team_fd'].tolist():
        psy_cursor.execute("""
                        INSERT INTO dwa.d_teams
                        VALUES (%s, %s, NULL,  NULL, NULL);                        
                        """
                        ,(team['hometeam'].strip().lower(), team['hometeam']))  
        aantal_nieuw += 1 

print(aantal_nieuw)

# Commit de transactie en sluit de verbinding
sqal_connection.commit()
psy_connection.commit()
psy_cursor.close()
sqal_connection.close()
psy_connection.close() 

        hometeam
0     Heidenheim
1         Empoli
2         Rennes
3         Alaves
4    Real Madrid
..           ...
349      Apollon
350   Schalke 04
351     Freiburg
352       Nantes
353  Giresunspor

[354 rows x 1 columns]
['Nancy', 'Man City', 'Levante', 'QPR', 'West Ham', 'Monaco', 'Carpi', 'Feirense', 'Almeria', 'Newcastle', 'Dordrecht', 'La Coruna', 'Kasimpasa', 'Heidenheim', 'Maritimo', 'Belenenses', 'Olympiakos', 'Spal', 'Balikesirspor', 'Arles', 'Den Haag', 'Evian Thonon Gaillard', 'Utrecht ', 'Sp Gijon', 'Cambuur', 'VVV Venlo', 'Pacos Ferreira', 'Antwerp', 'Heracles ', 'Karagumruk', 'Bastia', 'Graafschap', 'Goztep', 'AEK', 'Bari', 'Sp Lisbon', 'Volendam', 'Stuttgart', 'Lens', 'Lamia', 'Kallonis', 'Olympiakos Volou', 'Elazigspor', 'Penafiel', 'Santander', 'Spezia', 'Willem II ', 'Apollon Apollon', 'Bielefeld', 'Brescia', 'Hertha', 'Erciyesspor', 'Seraing', 'Reading', 'Genclerbirligi', 'Denizlispor', 'Karabukspor', 'Palermo', 'Zaragoza', 'Tottenham', 'Birmingham', 'Dijon', '

    Onderstaande code check teams in ODDAPI die overeenkomen met sleutel in teamtabel en voegt ze automatisch toe
    Daarna functie voor handmatig toevoegen van teams als teamnaam niet overeenkomst met sleutel uit teamtabel

In [4]:
import requests
import pandas as pd
import sqlalchemy
import psycopg2
import datetime

# connection variables
URL = 'football-fortunes.cj5o1qrvsyia.eu-west-1.rds.amazonaws.com'
database_name = 'postgres'
port = 5432
username = 'master'
ww = 'wachtwoord'

# Create SQL Alchemy database connection
db_url = f"postgresql://{username}:{ww}@{URL}:{port}/{database_name}"
engine = sqlalchemy.create_engine(db_url)
sqal_connection = engine.connect()

# Create Psychop connection
psy_connection = psycopg2.connect(
    host = URL,
    port = port,
    database = database_name,
    user= username,
    password = ww
)
psy_cursor = psy_connection.cursor()

# Haal alle teams op in fd_stats en in oddapi, alsmede de teams die al bestaan in tabel teams
teams_oddapi_odds = pd.read_sql("""SELECT distinct(hometeam) 
                                    FROM dsa.oddapi_odds
                                    UNION
                                    SELECT distinct(awayteam)
                                    FROM dsa.oddapi_odds
                                ;""", sqal_connection)
bestaande_teams = pd.read_sql('SELECT LOWER(team) as team, LOWER(team_oddapi) as team_oddapi FROM dwa.d_teams;', sqal_connection)

# Check of teams uit oddapi_odds voorkomen in teams tabel, kolom odd_api
# zo niet, check of naam overeenkomt met teamsleutel.
# zo ja, automatisch invoeren (en UPDATEN)
# zo nee, zie volgende stap
aantal_automatisch_toegevoegd = 0
teams_oddapi_lijst = []
for i, team in teams_oddapi_odds.iterrows():
    teamnaam = team['hometeam']
    if teamnaam.lower() not in bestaande_teams['team_oddapi'].tolist():
        if teamnaam.lower() in bestaande_teams['team'].tolist():
            teams_oddapi_lijst.append((teamnaam))
            aantal_automatisch_toegevoegd += 1

print(teams_oddapi_lijst)
for i in teams_oddapi_lijst:
    psy_cursor.execute("""
            UPDATE dwa.d_teams
            SET team_oddapi = %s
            WHERE team = %s;                      
            """,
            (i, i.lower()))

print(f'aantal automatisch toegevoegd: {aantal_automatisch_toegevoegd}')

# Haal alle teams op in fd_stats en in oddapi, alsmede de teams die al bestaan in tabel teams
teams_oddapi_odds = pd.read_sql("""SELECT distinct(hometeam) 
                                    FROM dsa.oddapi_odds
                                    UNION
                                    SELECT distinct(awayteam)
                                    FROM dsa.oddapi_odds
                                ;""", sqal_connection)
bestaande_teams = pd.read_sql('SELECT LOWER(team) as team, LOWER(team_oddapi) as team_oddapi FROM dwa.d_teams;', sqal_connection)

################################################################################################
# Check of teams uit oddapi_odds voorkomen en zo niet voer ze in
# invoer door eerste te checken of team te vinden is in sleutel
# zo nee, handmatig invoeren
aantal_handmatig_toegevoegd = 0
teams_oddapi_lijst = []
for i, team in teams_oddapi_odds.iterrows():
    teamnaam = team['hometeam']
    if teamnaam.lower() not in bestaande_teams['team_oddapi'].tolist():
        zoek_op = input(f'voor {teamnaam} zoek op:')
        suggestie = bestaande_teams[bestaande_teams['team'].str.contains(zoek_op)]
        print(suggestie)
        if teamnaam.lower() not in bestaande_teams['team'].tolist():
            team_sleutel = input(f"teamsleutel team_oddapi {teamnaam}").lower()
            psy_cursor.execute("""
                UPDATE dwa.d_teams
                SET team_oddapi = %s
                WHERE team = %s;                      
                """,
                (teamnaam, team_sleutel))
            psy_connection.commit()
            break

# update upate_tabel
psy_cursor.execute("""
                        UPDATE dsa.update
                        SET update_datum = current_date
                        WHERE tabelnaam = 'd_wedkantoren';                      
                        """)

# Commit de transactie en sluit de verbinding
sqal_connection.commit()
psy_connection.commit()
psy_cursor.close()
sqal_connection.close()
psy_connection.close() 

[]
aantal automatisch toegevoegd: 0
        team team_oddapi
88  kortrijk        None
